# Loading everything

In [151]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from statistics import mean

In [152]:
batch1 = pickle.load(open(r'batch1.pkl', 'rb'))
#remove batteries that do not reach 80% capacity
del batch1['b1c8']
del batch1['b1c10']
del batch1['b1c12']
del batch1['b1c13']
del batch1['b1c22']

In [153]:
numBat1 = len(batch1.keys())
numBat1

41

In [154]:
batch2 = pickle.load(open(r'batch2.pkl','rb'))

In [155]:
# There are four cells from batch1 that carried into batch2, we'll remove the data from batch2
# and put it with the correct cell from batch1
batch2_keys = ['b2c7', 'b2c8', 'b2c9', 'b2c15', 'b2c16']
batch1_keys = ['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4']
add_len = [662, 981, 1060, 208, 482];

In [156]:
for i, bk in enumerate(batch1_keys):
    batch1[bk]['cycle_life'] = batch1[bk]['cycle_life'] + add_len[i]
    for j in batch1[bk]['summary'].keys():
        if j == 'cycle':
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j] + len(batch1[bk]['summary'][j])))
        else:
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j]))
    last_cycle = len(batch1[bk]['cycles'].keys())
    for j, jk in enumerate(batch2[batch2_keys[i]]['cycles'].keys()):
        batch1[bk]['cycles'][str(last_cycle + j)] = batch2[batch2_keys[i]]['cycles'][jk]

In [157]:
del batch2['b2c7']
del batch2['b2c8']
del batch2['b2c9']
del batch2['b2c15']
del batch2['b2c16']

In [158]:
numBat2 = len(batch2.keys())
numBat2

43

In [159]:
batch3 = pickle.load(open(r'batch3.pkl','rb'))
# remove noisy channels from batch3
del batch3['b3c37']
del batch3['b3c2']
del batch3['b3c23']
del batch3['b3c32']
del batch3['b3c42']
del batch3['b3c43']

In [160]:
numBat3 = len(batch3.keys())
numBat3

40

In [161]:
numBat = numBat1 + numBat2 + numBat3
numBat

124

In [162]:
bat_dict = {**batch1, **batch2, **batch3}

In [163]:
for i in bat_dict.keys():
    print(i)

b1c0
b1c1
b1c2
b1c3
b1c4
b1c5
b1c6
b1c7
b1c9
b1c11
b1c14
b1c15
b1c16
b1c17
b1c18
b1c19
b1c20
b1c21
b1c23
b1c24
b1c25
b1c26
b1c27
b1c28
b1c29
b1c30
b1c31
b1c32
b1c33
b1c34
b1c35
b1c36
b1c37
b1c38
b1c39
b1c40
b1c41
b1c42
b1c43
b1c44
b1c45
b2c0
b2c1
b2c2
b2c3
b2c4
b2c5
b2c6
b2c10
b2c11
b2c12
b2c13
b2c14
b2c17
b2c18
b2c19
b2c20
b2c21
b2c22
b2c23
b2c24
b2c25
b2c26
b2c27
b2c28
b2c29
b2c30
b2c31
b2c32
b2c33
b2c34
b2c35
b2c36
b2c37
b2c38
b2c39
b2c40
b2c41
b2c42
b2c43
b2c44
b2c45
b2c46
b2c47
b3c0
b3c1
b3c3
b3c4
b3c5
b3c6
b3c7
b3c8
b3c9
b3c10
b3c11
b3c12
b3c13
b3c14
b3c15
b3c16
b3c17
b3c18
b3c19
b3c20
b3c21
b3c22
b3c24
b3c25
b3c26
b3c27
b3c28
b3c29
b3c30
b3c31
b3c33
b3c34
b3c35
b3c36
b3c38
b3c39
b3c40
b3c41
b3c44
b3c45


# Getting all the features

In [164]:
from scipy.stats import skew, kurtosis
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np

In [165]:
features_df = pd.DataFrame()

In [166]:
minimum_dQ_100_10 = np.zeros(len(bat_dict.keys()))
variance_dQ_100_10 = np.zeros(len(bat_dict.keys()))
skewness_dQ_100_10 = np.zeros(len(bat_dict.keys()))
kurtosis_dQ_100_10 = np.zeros(len(bat_dict.keys()))

for i, cell in enumerate(bat_dict.values()):
    c10 = cell['cycles']['10']
    c100 = cell['cycles']['100']
    dQ_100_10 = c100['Qdlin'] - c10['Qdlin']
    
    minimum_dQ_100_10[i] = np.log(np.abs(np.min(dQ_100_10)))
    variance_dQ_100_10[i] = np.log(np.var(dQ_100_10))
    skewness_dQ_100_10[i] = np.log(np.abs(skew(dQ_100_10)))
    kurtosis_dQ_100_10[i] = np.log(np.abs(kurtosis(dQ_100_10)))

features_df["minimum_dQ_100_10"] = minimum_dQ_100_10
features_df["variance_dQ_100_10"] = variance_dQ_100_10
features_df["skewness_dQ_100_10"] = skewness_dQ_100_10
features_df["kurtosis_dQ_100_10"] = kurtosis_dQ_100_10


slope_lin_fit_2_100 = np.zeros(len(bat_dict.keys()))
intercept_lin_fit_2_100 = np.zeros(len(bat_dict.keys()))
discharge_capacity_2 = np.zeros(len(bat_dict.keys()))
diff_discharge_capacity_max_2 = np.zeros(len(bat_dict.keys()))

for i, cell in enumerate(bat_dict.values()):
    # Compute linear fit for cycles 2 to 100:
    q = cell['summary']['QD'][1:100].reshape(-1, 1)  # discharge cappacities; q.shape = (99, 1); 
    X = cycle_numbers = cell['summary']['cycle'][1:100].reshape(-1, 1)  # Cylce index from 2 to 100; X.shape = (99, 1)
    
    linear_regressor_2_100 = LinearRegression()
    linear_regressor_2_100.fit(X, q)
    
    slope_lin_fit_2_100[i] = linear_regressor_2_100.coef_[0]
    intercept_lin_fit_2_100[i] = linear_regressor_2_100.intercept_
    discharge_capacity_2[i] = q[0][0]
    diff_discharge_capacity_max_2[i] = np.max(q) - q[0][0]
    
features_df["slope_lin_fit_2_100"] = slope_lin_fit_2_100
features_df["intercept_lin_fit_2_100"] = intercept_lin_fit_2_100
features_df["discharge_capacity_2"] = discharge_capacity_2
features_df["diff_discharge_capacity_max_2"] = diff_discharge_capacity_max_2

mean_charge_time = np.zeros(len(bat_dict.keys()))
minimum_IR = np.zeros(len(bat_dict.keys()))
diff_IR_100_2 = np.zeros(len(bat_dict.keys()))


for i, cell in enumerate(bat_dict.values()):
    mean_charge_time[i] = np.mean(cell['summary']['chargetime'][1:6])
    minimum_IR[i] = np.min(cell['summary']['IR'][1:100])
    diff_IR_100_2[i] = cell['summary']['IR'][100] - cell['summary']['IR'][1]
    
features_df["mean_charge_time"] = mean_charge_time
features_df["minimum_IR"] = minimum_IR
features_df["diff_IR_100_2"] = diff_IR_100_2

In [167]:
features_df.head()

,minimum_dQ_100_10,variance_dQ_100_10,skewness_dQ_100_10,kurtosis_dQ_100_10,slope_lin_fit_2_100,intercept_lin_fit_2_100,discharge_capacity_2,diff_discharge_capacity_max_2,mean_charge_time,minimum_IR,diff_IR_100_2
0,-4.720901,-11.631570,-0.319238,-0.094314,-0.000207,1.091144,1.070689,0.468365,13.374894,0.016444,-0.000092
1,-4.709132,-11.825562,-1.148991,0.037823,0.000006,1.080965,1.075301,0.009288,13.409150,0.000000,-0.000034
2,-4.575223,-11.402131,-0.826717,0.160194,0.000010,1.084846,1.079922,0.008131,13.358242,0.000000,-0.000009
3,-3.922042,-10.099939,-0.741646,0.117089,0.000017,1.084075,1.079723,0.006530,12.025140,0.000000,0.000044
4,-4.230762,-10.602896,-1.052154,0.308340,0.000019,1.081875,1.078417,0.005900,12.041851,0.000000,-0.000036


In [168]:
features_df.shape

(124, 11)

In [169]:
cols = features_df.columns

# Splitting the data 80-20

In [170]:
target_df = pd.DataFrame()
cycle_life = []
for i in bat_dict:
    cycle_life.append(np.float(bat_dict[i]["cycle_life"]))
cycle_life = pd.Series(cycle_life)
target_df["cycle_life"] = cycle_life
target_df.head()

,cycle_life
0,1852.0
1,2160.0
2,2237.0
3,1434.0
4,1709.0


In [171]:
x = features_df
y = cycle_life

In [172]:
import numpy as np
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)

In [173]:
x_train

,minimum_dQ_100_10,variance_dQ_100_10,skewness_dQ_100_10,kurtosis_dQ_100_10,slope_lin_fit_2_100,intercept_lin_fit_2_100,discharge_capacity_2,diff_discharge_capacity_max_2,mean_charge_time,minimum_IR,diff_IR_100_2
0,-4.720901,-11.631570,-0.319238,-0.094314,-0.000207,1.091144,1.070689,0.468365,13.374894,0.016444,-0.000092
1,-4.709132,-11.825562,-1.148991,0.037823,0.000006,1.080965,1.075301,0.009288,13.409150,0.000000,-0.000034
2,-4.575223,-11.402131,-0.826717,0.160194,0.000010,1.084846,1.079922,0.008131,13.358242,0.000000,-0.000009
3,-3.922042,-10.099939,-0.741646,0.117089,0.000017,1.084075,1.079723,0.006530,12.025140,0.000000,0.000044
4,-4.230762,-10.602896,-1.052154,0.308340,0.000019,1.081875,1.078417,0.005900,12.041851,0.000000,-0.000036
...,...,...,...,...,...,...,...,...,...,...,...
94,-3.388775,-8.972545,-2.071800,0.225442,-0.000018,1.070729,1.068068,0.002693,10.042218,0.015485,-0.000050
95,-3.442255,-9.071859,-5.037440,0.243642,-0.000007,1.076676,1.074165,0.002778,10.042492,0.015998,-0.000024
96,-3.584359,-9.417628,-1.439153,0.166120,-0.000012,1.069335,1.065558,0.004087,10.042320,0.015064,-0.000150
97,-3.530465,-9.245718,-1.693350,0.254449,0.000005,1.076779,1.072706,0.005039,10.041712,0.015497,-0.000069


In [174]:
x_test

,minimum_dQ_100_10,variance_dQ_100_10,skewness_dQ_100_10,kurtosis_dQ_100_10,slope_lin_fit_2_100,intercept_lin_fit_2_100,discharge_capacity_2,diff_discharge_capacity_max_2,mean_charge_time,minimum_IR,diff_IR_100_2
99,-4.279456,-10.787515,-2.117039,-0.016650,-9.691301e-06,1.064003,1.061009,0.003331,11.043366,0.015253,-0.000120
100,-3.995719,-10.211420,-3.319977,-0.083643,-5.833126e-06,1.067345,1.066451,0.002600,10.041974,0.015354,-0.000149
101,-3.745685,-9.754075,-1.543652,0.063067,-2.927494e-05,1.070362,1.068024,0.002130,10.043130,0.014978,-0.000049
102,-3.656142,-9.560392,-2.124784,0.110625,-2.969136e-05,1.074235,1.071925,0.001975,10.043437,0.015178,-0.000042
103,-3.598892,-9.432141,-1.167912,0.159191,-7.154074e-06,1.067815,1.065050,0.003061,10.043330,0.015207,-0.000084
104,-3.121523,-8.503500,-0.683385,-0.172339,-1.594917e-04,1.070112,1.067856,0.000840,10.067868,0.013940,0.000247
105,-3.566776,-9.315545,-2.477689,0.231141,-1.598876e-05,1.058970,1.056264,0.002845,10.043384,0.015615,-0.000120
106,-3.672624,-9.502554,-1.149260,0.241388,9.650589e-06,1.049949,1.046273,0.004856,10.042240,0.015503,-0.000152
107,-3.616309,-9.514009,-1.122250,0.105527,6.950844e-06,1.064468,1.061130,0.004340,10.041598,0.015591,-0.000038
108,-3.706722,-9.692178,-1.167855,0.079232,-4.288423e-07,1.075134,1.072019,0.003791,10.041817,0.015539,-0.000050


# APPROACH 1

## VARIANCE

In [231]:
error_list = []
real_lifetime_list = []
predicted_lifetime_list = []
cell_number_train = []
variance_test = []
variance_train = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    df["cell_test"] = pd.Series()
    df["cell_train"] = pd.Series()
    df["distance"] = pd.Series()
    cell_test = []
    cell_train = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test.append(x_test["variance_dQ_100_10"][i])
    for a in x_train.index:
        cell_train.append(x_train["variance_dQ_100_10"][a])
    df["cell_test"] = pd.Series(cell_test)
    df["cell_train"] = pd.Series(cell_train)
    df["distance"] = abs(df["cell_test"] - df["cell_train"])

    #finding the most similar cell
    minimum = min(df["distance"])
    cell_number_train.append(df["cell_train"].loc[df["distance"]==minimum].index[0])
    number = df["cell_train"].loc[df["distance"]==minimum].index[0]
    #append features to csv
    variance_test.append(x_test["variance_dQ_100_10"][i])
    variance_train.append(x_train["variance_dQ_100_10"][number])

    #calculating the error
    real_lifetime = y_test[i]
    real_lifetime_list.append(float(real_lifetime))

    predicted_lifetime = y_train[(df.loc[df['distance'] == minimum].index)]
    predicted_lifetime_list.append(float(predicted_lifetime))

    error = float(abs(real_lifetime - predicted_lifetime))
    error = error/real_lifetime
    error_list.append(error*100)


    #print
    print(df)
    print("Error: ", np.array(error))

#creating CSV df
csv_df = pd.DataFrame()
csv_df["cell_number_test"] = pd.Series(list(x_test.index))
csv_df["cell_number_train"] = pd.Series(cell_number_train)
csv_df["variance_test"] = pd.Series(variance_test)
csv_df["variance_train"] = pd.Series(variance_train)
csv_df["prediction"] = pd.Series(predicted_lifetime_list)
csv_df["real"] = pd.Series(real_lifetime_list)
csv_df["error"] = pd.Series(error_list)   
csv_df.to_csv('variance_app1.csv') 

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  del sys.path[0]
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/si

    cell_test  cell_train  distance
0  -10.787515  -11.631570  0.844054
1  -10.787515  -11.825562  1.038046
2  -10.787515  -11.402131  0.614616
3  -10.787515  -10.099939  0.687576
4  -10.787515  -10.602896  0.184619
..        ...         ...       ...
94 -10.787515   -8.972545  1.814970
95 -10.787515   -9.071859  1.715656
96 -10.787515   -9.417628  1.369887
97 -10.787515   -9.245718  1.541797
98 -10.787515   -9.190990  1.596525

[99 rows x 3 columns]
Error:  0.043345543345543344
    cell_test  cell_train  distance
0   -10.21142  -11.631570  1.420149
1   -10.21142  -11.825562  1.614141
2   -10.21142  -11.402131  1.190711
3   -10.21142  -10.099939  0.111482
4   -10.21142  -10.602896  0.391476
..        ...         ...       ...
94  -10.21142   -8.972545  1.238875
95  -10.21142   -9.071859  1.139561
96  -10.21142   -9.417628  0.793792
97  -10.21142   -9.245718  0.965702
98  -10.21142   -9.190990  1.020430

[99 rows x 3 columns]
Error:  0.18022813688212927
    cell_test  cell_train  distan

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  del sys.path[0]
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/si

In [176]:
pd.Series(error_list).describe()

count    25.000000
mean     14.332477
std      10.960002
min       1.845018
25%       6.747405
50%      11.954625
75%      18.617021
max      35.151515
dtype: float64

## DISCHARGE

In [232]:
error_list = []
real_lifetime_list = []
predicted_lifetime_list = []
cell_number_train = []
variance_test = []
variance_train = []
minimum_test = []
minimum_train = []
skewness_test = []
skewness_train = []
kurtosis_test = []
kurtosis_train = []
dc2_test = []
dc2_train = []
ddc2_test = []
ddc2_train = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #skewness
    df["cell_test_skewness"] = pd.Series()
    df["cell_train_skewness"] = pd.Series()
    #kurtosis
    df["cell_test_kurtosis"] = pd.Series()
    df["cell_train_kurtosis"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #diff discharge capacity max 2
    df["cell_test_ddc2"] = pd.Series()
    df["cell_train_ddc2"] = pd.Series()
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_skewness = []
    cell_test_kurtosis = []
    cell_test_dc2 = []
    cell_test_ddc2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_skewness = []
    cell_train_kurtosis = []
    cell_train_dc2 = []
    cell_train_ddc2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_skewness.append(x_test["skewness_dQ_100_10"][i])
        cell_test_kurtosis.append(x_test["kurtosis_dQ_100_10"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_ddc2.append(x_test["diff_discharge_capacity_max_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_skewness.append(x_train["skewness_dQ_100_10"][a])
        cell_train_kurtosis.append(x_train["kurtosis_dQ_100_10"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_ddc2.append(x_train["diff_discharge_capacity_max_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_skewness"] = pd.Series(cell_test_skewness)
    df["cell_test_kurtosis"] = pd.Series(cell_test_kurtosis)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_ddc2"] = pd.Series(cell_test_ddc2)

    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_skewness"] = pd.Series(cell_train_skewness)
    df["cell_train_kurtosis"] = pd.Series(cell_train_kurtosis)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_ddc2"] = pd.Series(cell_train_ddc2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_skewness"] - df["cell_train_skewness"])**2
    d = (df["cell_test_kurtosis"] - df["cell_train_kurtosis"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_ddc2"] - df["cell_train_ddc2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f)

    #finding the most similar cell
    minimum = min(df["distance"])
    cell_number_train.append(df["cell_train_variance"].loc[df["distance"]==minimum].index[0])
    number = df["cell_train_variance"].loc[df["distance"]==minimum].index[0]
    #append features to csv
    variance_test.append(x_test["variance_dQ_100_10"][i])
    variance_train.append(x_train["variance_dQ_100_10"][number])
    minimum_test.append(x_test["variance_dQ_100_10"][i])
    minimum_train.append(x_train["variance_dQ_100_10"][number])
    skewness_test.append(x_test["variance_dQ_100_10"][i])
    skewness_train.append(x_train["variance_dQ_100_10"][number])
    kurtosis_test.append(x_test["variance_dQ_100_10"][i])
    kurtosis_train.append(x_train["variance_dQ_100_10"][number])
    dc2_test.append(x_test["variance_dQ_100_10"][i])
    dc2_train.append(x_train["variance_dQ_100_10"][number])
    ddc2_test.append(x_test["variance_dQ_100_10"][i])
    ddc2_train.append(x_train["variance_dQ_100_10"][number])
    
    #calculating the error
    real_lifetime = y_test[i]
    real_lifetime_list.append(float(real_lifetime))

    predicted_lifetime = y_train[df.loc[df['distance'] == minimum].index]
    predicted_lifetime_list.append(float(predicted_lifetime))

    error = float(abs(real_lifetime - predicted_lifetime))
    error = error/real_lifetime
    error_list.append(error*100)

    #print
    print(df)
    print("Error: ", np.array(error))

#creating CSV df
csv_df = pd.DataFrame()
csv_df["cell_number_test"] = pd.Series(list(x_test.index))
csv_df["cell_number_train"] = pd.Series(cell_number_train)
csv_df["variance_test"] = pd.Series(variance_test)
csv_df["variance_train"] = pd.Series(variance_train)
csv_df["minimum_test"] = pd.Series(minimum_test)
csv_df["minimum_train"] = pd.Series(minimum_train)
csv_df["skewness_test"] = pd.Series(skewness_test)
csv_df["skewness_train"] = pd.Series(skewness_train)
csv_df["kurtosis_test"] = pd.Series(kurtosis_test)
csv_df["kurtosis_train"] = pd.Series(kurtosis_train)
csv_df["dc2_test"] = pd.Series(dc2_test)
csv_df["dc2_train"] = pd.Series(dc2_train)
csv_df["ddc2_test"] = pd.Series(ddc2_test)
csv_df["ddc2_test"] = pd.Series(ddc2_test)
csv_df["prediction"] = pd.Series(predicted_lifetime_list)
csv_df["real"] = pd.Series(real_lifetime_list)
csv_df["error"] = pd.Series(error_list)
csv_df.to_csv('discharge_app1.csv') 

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.787515           -11.631570          -4.279456   
1           -10.787515           -11.825562          -4.279456   
2           -10.787515           -11.402131          -4.279456   
3           -10.787515           -10.099939          -4.279456   
4           -10.787515           -10.602896          -4.279456   
..                 ...                  ...                ...   
94          -10.787515            -8.972545          -4.279456   
95          -10.787515            -9.071859          -4.279456   
96          -10.787515            -9.417628          -4.279456   
97          -10.787515            -9.245718          -4.279456   
98          -10.787515            -9.190990          -4.279456   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -2.117039            -0.319238   
1            -4.709132           -2.117039            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.315545           -11.631570          -3.566776   
1            -9.315545           -11.825562          -3.566776   
2            -9.315545           -11.402131          -3.566776   
3            -9.315545           -10.099939          -3.566776   
4            -9.315545           -10.602896          -3.566776   
..                 ...                  ...                ...   
94           -9.315545            -8.972545          -3.566776   
95           -9.315545            -9.071859          -3.566776   
96           -9.315545            -9.417628          -3.566776   
97           -9.315545            -9.245718          -3.566776   
98           -9.315545            -9.190990          -3.566776   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -2.477689            -0.319238   
1            -4.709132           -2.477689            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.540689           -11.631570          -3.651003   
1            -9.540689           -11.825562          -3.651003   
2            -9.540689           -11.402131          -3.651003   
3            -9.540689           -10.099939          -3.651003   
4            -9.540689           -10.602896          -3.651003   
..                 ...                  ...                ...   
94           -9.540689            -8.972545          -3.651003   
95           -9.540689            -9.071859          -3.651003   
96           -9.540689            -9.417628          -3.651003   
97           -9.540689            -9.245718          -3.651003   
98           -9.540689            -9.190990          -3.651003   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -1.117463            -0.319238   
1            -4.709132           -1.117463            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be


Error:  0.5545219638242894
    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.230704           -11.631570          -4.031907   
1           -10.230704           -11.825562          -4.031907   
2           -10.230704           -11.402131          -4.031907   
3           -10.230704           -10.099939          -4.031907   
4           -10.230704           -10.602896          -4.031907   
..                 ...                  ...                ...   
94          -10.230704            -8.972545          -4.031907   
95          -10.230704            -9.071859          -4.031907   
96          -10.230704            -9.417628          -4.031907   
97          -10.230704            -9.245718          -4.031907   
98          -10.230704            -9.190990          -4.031907   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -1.208466            -0.319238   
1            -4.709132           -1.208466   

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be

In [178]:
pd.Series(error_list).describe()

count    50.000000
mean     17.116130
std      13.151568
min       1.845018
25%       7.814902
50%      13.221385
75%      25.019276
max      58.823529
dtype: float64

## FULL MODEL

In [233]:
error_list = []
real_lifetime_list = []
predicted_lifetime_list = []
cell_number_train = []
variance_test = []
variance_train = []
minimum_test = []
minimum_train = []
slope_lin_fit_2_100_test = []
slope_lin_fit_2_100_train = []
intercept_lin_fit_2_100_test = []
intercept_lin_fit_2_100_train = []
dc2_test = []
dc2_train = []
mean_charge_time_test = []
mean_charge_time_train = []
minimumIR_test = []
minimumIR_train = []
diff_IR_100_2_test = []
diff_IR_100_2_train = []



for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #slope_lin_fit_2_100
    df["cell_test_slope_lin_fit_2_100"] = pd.Series()
    df["cell_train_slope_lin_fit_2_100"] = pd.Series()
    #intercept_lin_fit_2_100
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series()
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #mean charge time
    df["cell_test_mean_charge_time"] = pd.Series()
    df["cell_train_mean_charge_time"] = pd.Series()  
    #minimum IR
    df["cell_test_minimum_IR"] = pd.Series()
    df["cell_train_minimum_IR"] = pd.Series()
    #diff_IR_100_2
    df["cell_test_diff_IR_100_2"] = pd.Series()
    df["cell_train_diff_IR_100_2"] = pd.Series()  
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_slope_lin_fit_2_100 = []
    cell_test_intercept_lin_fit_2_100 = []
    cell_test_dc2 = []
    cell_test_mean_charge_time = []
    cell_test_minimum_IR = []
    cell_test_diff_IR_100_2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_slope_lin_fit_2_100 = []
    cell_train_intercept_lin_fit_2_100 = []
    cell_train_dc2 = []
    cell_train_mean_charge_time = []
    cell_train_minimum_IR = []
    cell_train_diff_IR_100_2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_slope_lin_fit_2_100.append(x_test["slope_lin_fit_2_100"][i])
        cell_test_intercept_lin_fit_2_100.append(x_test["intercept_lin_fit_2_100"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_mean_charge_time.append(x_test["mean_charge_time"][i])
        cell_test_minimum_IR.append(x_test["minimum_IR"][i])
        cell_test_diff_IR_100_2.append(x_test["diff_IR_100_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_slope_lin_fit_2_100.append(x_train["slope_lin_fit_2_100"][a])
        cell_train_intercept_lin_fit_2_100.append(x_train["intercept_lin_fit_2_100"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_mean_charge_time.append(x_train["mean_charge_time"][a])
        cell_train_minimum_IR.append(x_train["minimum_IR"][a])
        cell_train_diff_IR_100_2.append(x_train["diff_IR_100_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_slope_lin_fit_2_100"] = pd.Series(cell_test_slope_lin_fit_2_100)
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series(cell_test_intercept_lin_fit_2_100)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_mean_charge_time"] = pd.Series(cell_test_mean_charge_time)
    df["cell_test_minimum_IR"] = pd.Series(cell_test_minimum_IR)
    df["cell_test_diff_IR_100_2"] = pd.Series(cell_test_diff_IR_100_2)


    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_slope_lin_fit_2_100"] = pd.Series(cell_train_slope_lin_fit_2_100)
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series(cell_train_intercept_lin_fit_2_100)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_mean_charge_time"] = pd.Series(cell_train_mean_charge_time)
    df["cell_train_minimum_IR"] = pd.Series(cell_train_minimum_IR)
    df["cell_train_diff_IR_100_2"] = pd.Series(cell_train_diff_IR_100_2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_slope_lin_fit_2_100"] - df["cell_train_slope_lin_fit_2_100"])**2
    d = (df["cell_test_intercept_lin_fit_2_100"] - df["cell_train_intercept_lin_fit_2_100"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_mean_charge_time"] - df["cell_train_mean_charge_time"])**2
    g = (df["cell_test_minimum_IR"] - df["cell_train_minimum_IR"])**2
    h = (df["cell_test_diff_IR_100_2"] - df["cell_train_diff_IR_100_2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f+g+h)

    #finding the most similar cell
    minimum = min(df["distance"])
    cell_number_train.append(df["cell_train_variance"].loc[df["distance"]==minimum].index[0])
    number = df["cell_train_variance"].loc[df["distance"]==minimum].index[0]
    #append features to csv
    variance_test.append(x_test["variance_dQ_100_10"][i])
    variance_train.append(x_train["variance_dQ_100_10"][number])
    minimum_test.append(x_test["variance_dQ_100_10"][i])
    minimum_train.append(x_train["variance_dQ_100_10"][number])
    slope_lin_fit_2_100_test.append(x_test["variance_dQ_100_10"][i])
    slope_lin_fit_2_100_train.append(x_train["variance_dQ_100_10"][number])
    intercept_lin_fit_2_100_test.append(x_test["variance_dQ_100_10"][i])
    intercept_lin_fit_2_100_train.append(x_train["variance_dQ_100_10"][number])
    dc2_test.append(x_test["variance_dQ_100_10"][i])
    dc2_train.append(x_train["variance_dQ_100_10"][number])
    mean_charge_time_test.append(x_test["variance_dQ_100_10"][i])
    mean_charge_time_train.append(x_train["variance_dQ_100_10"][number])
    minimumIR_test.append(x_test["variance_dQ_100_10"][i])
    minimumIR_train.append(x_train["variance_dQ_100_10"][number])
    diff_IR_100_2_test.append(x_test["variance_dQ_100_10"][i])
    diff_IR_100_2_train.append(x_train["variance_dQ_100_10"][number])

    #calculating the error
    real_lifetime = y_test[i]
    real_lifetime_list.append(float(real_lifetime))

    predicted_lifetime = y_train[df.loc[df['distance'] == minimum].index]
    predicted_lifetime_list.append(float(predicted_lifetime))

    error = float(abs(real_lifetime - predicted_lifetime))
    error = error/real_lifetime
    error_list.append(error*100)

    #print
    print(df)
    print("Error: ", np.array(error))

#creating CSV df
csv_df = pd.DataFrame()
csv_df["cell_number_test"] = pd.Series(list(x_test.index))
csv_df["cell_number_train"] = pd.Series(cell_number_train)
csv_df["variance_test"] = pd.Series(variance_test)
csv_df["variance_train"] = pd.Series(variance_train)
csv_df["minimum_test"] = pd.Series(minimum_test)
csv_df["minimum_train"] = pd.Series(minimum_train)
csv_df["slope_lin_fit_2_100_test"] = pd.Series(slope_lin_fit_2_100_test)
csv_df["slope_lin_fit_2_100_train"] = pd.Series(slope_lin_fit_2_100_train)
csv_df["intercept_lin_fit_2_100_test"] = pd.Series(intercept_lin_fit_2_100_test)
csv_df["intercept_lin_fit_2_100_train"] = pd.Series(intercept_lin_fit_2_100_train)
csv_df["dc2_test"] = pd.Series(dc2_test)
csv_df["dc2_train"] = pd.Series(dc2_train)
csv_df["mean_charge_time_test"] = pd.Series(mean_charge_time_test)
csv_df["mean_charge_time_train"] = pd.Series(mean_charge_time_train)
csv_df["minimumIR_test"] = pd.Series(minimumIR_test)
csv_df["minimumIR_train"] = pd.Series(minimumIR_train)
csv_df["diff_IR_100_2_test"] = pd.Series(diff_IR_100_2_test)
csv_df["diff_IR_100_2_train"] = pd.Series(diff_IR_100_2_train)
csv_df["prediction"] = pd.Series(predicted_lifetime_list)
csv_df["real"] = pd.Series(real_lifetime_list)
csv_df["error"] = pd.Series(error_list)
csv_df.to_csv('fullmodel_app1.csv') 

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: The default dtype for empty Series will be

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.787515           -11.631570          -4.279456   
1           -10.787515           -11.825562          -4.279456   
2           -10.787515           -11.402131          -4.279456   
3           -10.787515           -10.099939          -4.279456   
4           -10.787515           -10.602896          -4.279456   
..                 ...                  ...                ...   
94          -10.787515            -8.972545          -4.279456   
95          -10.787515            -9.071859          -4.279456   
96          -10.787515            -9.417628          -4.279456   
97          -10.787515            -9.245718          -4.279456   
98          -10.787515            -9.190990          -4.279456   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                       -0.00001   
1            -4.709132                       -0.00001   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: The default dtype for empty Series will be


Error:  0.4326424870466321
    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.315545           -11.631570          -3.566776   
1            -9.315545           -11.825562          -3.566776   
2            -9.315545           -11.402131          -3.566776   
3            -9.315545           -10.099939          -3.566776   
4            -9.315545           -10.602896          -3.566776   
..                 ...                  ...                ...   
94           -9.315545            -8.972545          -3.566776   
95           -9.315545            -9.071859          -3.566776   
96           -9.315545            -9.417628          -3.566776   
97           -9.315545            -9.245718          -3.566776   
98           -9.315545            -9.190990          -3.566776   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000016   
1            -4.709132                      -0.000016   
2       

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: The default dtype for empty Series will be

...

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: The default dtype for empty Series will be


Error:  0.1721183800623053
    cell_test_variance  cell_train_variance  cell_test_minimum  \
0             -9.88592           -11.631570          -3.826262   
1             -9.88592           -11.825562          -3.826262   
2             -9.88592           -11.402131          -3.826262   
3             -9.88592           -10.099939          -3.826262   
4             -9.88592           -10.602896          -3.826262   
..                 ...                  ...                ...   
94            -9.88592            -8.972545          -3.826262   
95            -9.88592            -9.071859          -3.826262   
96            -9.88592            -9.417628          -3.826262   
97            -9.88592            -9.245718          -3.826262   
98            -9.88592            -9.190990          -3.826262   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000021   
1            -4.709132                      -0.000021   
2       

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:41: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: The default dtype for empty Series will be

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.230704           -11.631570          -4.031907   
1           -10.230704           -11.825562          -4.031907   
2           -10.230704           -11.402131          -4.031907   
3           -10.230704           -10.099939          -4.031907   
4           -10.230704           -10.602896          -4.031907   
..                 ...                  ...                ...   
94          -10.230704            -8.972545          -4.031907   
95          -10.230704            -9.071859          -4.031907   
96          -10.230704            -9.417628          -4.031907   
97          -10.230704            -9.245718          -4.031907   
98          -10.230704            -9.190990          -4.031907   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000007   
1            -4.709132                      -0.000007   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: The default dtype for empty Series will be

In [180]:
pd.Series(error_list).describe()

count    75.000000
mean     16.694216
std      13.059859
min       0.000000
25%       7.685270
50%      12.867012
75%      22.365954
max      58.823529
dtype: float64

# APPROACH 2 (3 most similar cells)

## VARIANCE

In [181]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    df["cell_test"] = pd.Series()
    df["cell_train"] = pd.Series()
    df["distance"] = pd.Series()
    cell_test = []
    cell_train = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test.append(x_test["variance_dQ_100_10"][i])
    for a in x_train.index:
        cell_train.append(x_train["variance_dQ_100_10"][a])
    df["cell_test"] = pd.Series(cell_test)
    df["cell_train"] = pd.Series(cell_train)
    df["distance"] = abs(df["cell_test"] - df["cell_train"])

    #finding the 3 most similar cells
    three_most_similar_cells = df.nsmallest(3, 'distance')
    number1 = int(y_train[three_most_similar_cells.iloc[0].name])
    number2 = int(y_train[three_most_similar_cells.iloc[1].name])
    number3 = int(y_train[three_most_similar_cells.iloc[2].name])

    #getting max and min
    list_of_three_most_similar_cells = [number1, number2, number3]
    max_predicted_lifetime = max(list_of_three_most_similar_cells)
    min_predicted_lifetime = min(list_of_three_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))

    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")
    
    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)


/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/e

    cell_test  cell_train  distance
0  -10.787515  -11.631570  0.844054
1  -10.787515  -11.825562  1.038046
2  -10.787515  -11.402131  0.614616
3  -10.787515  -10.099939  0.687576
4  -10.787515  -10.602896  0.184619
..        ...         ...       ...
94 -10.787515   -8.972545  1.814970
95 -10.787515   -9.071859  1.715656
96 -10.787515   -9.417628  1.369887
97 -10.787515   -9.245718  1.541797
98 -10.787515   -9.190990  1.596525

[99 rows x 3 columns]
Predicted lifetime range is from:  1709 to  2237
Actual lifetime:  1638


    cell_test  cell_train  distance
0   -10.21142  -11.631570  1.420149
1   -10.21142  -11.825562  1.614141
2   -10.21142  -11.402131  1.190711
3   -10.21142  -10.099939  0.111482
4   -10.21142  -10.602896  0.391476
..        ...         ...       ...
94  -10.21142   -8.972545  1.238875
95  -10.21142   -9.071859  1.139561
96  -10.21142   -9.417628  0.793792
97  -10.21142   -9.245718  0.965702
98  -10.21142   -9.190990  1.020430

[99 rows x 3 columns]
Predicted lifeti

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':


In [182]:
accuracy_list.count("No")

12

In [183]:
mean(range_list)

292

## DISCHARGE

In [184]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #skewness
    df["cell_test_skewness"] = pd.Series()
    df["cell_train_skewness"] = pd.Series()
    #kurtosis
    df["cell_test_kurtosis"] = pd.Series()
    df["cell_train_kurtosis"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #diff discharge capacity max 2
    df["cell_test_ddc2"] = pd.Series()
    df["cell_train_ddc2"] = pd.Series()
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_skewness = []
    cell_test_kurtosis = []
    cell_test_dc2 = []
    cell_test_ddc2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_skewness = []
    cell_train_kurtosis = []
    cell_train_dc2 = []
    cell_train_ddc2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_skewness.append(x_test["skewness_dQ_100_10"][i])
        cell_test_kurtosis.append(x_test["kurtosis_dQ_100_10"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_ddc2.append(x_test["diff_discharge_capacity_max_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_skewness.append(x_train["skewness_dQ_100_10"][a])
        cell_train_kurtosis.append(x_train["kurtosis_dQ_100_10"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_ddc2.append(x_train["diff_discharge_capacity_max_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_skewness"] = pd.Series(cell_test_skewness)
    df["cell_test_kurtosis"] = pd.Series(cell_test_kurtosis)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_ddc2"] = pd.Series(cell_test_ddc2)

    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_skewness"] = pd.Series(cell_train_skewness)
    df["cell_train_kurtosis"] = pd.Series(cell_train_kurtosis)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_ddc2"] = pd.Series(cell_train_ddc2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_skewness"] - df["cell_train_skewness"])**2
    d = (df["cell_test_kurtosis"] - df["cell_train_kurtosis"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_ddc2"] - df["cell_train_ddc2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f)

    #finding the 3 most similar cells
    three_most_similar_cells = df.nsmallest(3, 'distance')
    number1 = int(y_train[three_most_similar_cells.iloc[0].name])
    number2 = int(y_train[three_most_similar_cells.iloc[1].name])
    number3 = int(y_train[three_most_similar_cells.iloc[2].name])

    #getting max and min
    list_of_three_most_similar_cells = [number1, number2, number3]
    max_predicted_lifetime = max(list_of_three_most_similar_cells)
    min_predicted_lifetime = min(list_of_three_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

    
        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)


/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.787515           -11.631570          -4.279456   
1           -10.787515           -11.825562          -4.279456   
2           -10.787515           -11.402131          -4.279456   
3           -10.787515           -10.099939          -4.279456   
4           -10.787515           -10.602896          -4.279456   
..                 ...                  ...                ...   
94          -10.787515            -8.972545          -4.279456   
95          -10.787515            -9.071859          -4.279456   
96          -10.787515            -9.417628          -4.279456   
97          -10.787515            -9.245718          -4.279456   
98          -10.787515            -9.190990          -4.279456   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -2.117039            -0.319238   
1            -4.709132           -2.117039            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.502554           -11.631570          -3.672624   
1            -9.502554           -11.825562          -3.672624   
2            -9.502554           -11.402131          -3.672624   
3            -9.502554           -10.099939          -3.672624   
4            -9.502554           -10.602896          -3.672624   
..                 ...                  ...                ...   
94           -9.502554            -8.972545          -3.672624   
95           -9.502554            -9.071859          -3.672624   
96           -9.502554            -9.417628          -3.672624   
97           -9.502554            -9.245718          -3.672624   
98           -9.502554            -9.190990          -3.672624   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901            -1.14926            -0.319238   
1            -4.709132            -1.14926            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.063654           -11.631570          -3.886203   
1           -10.063654           -11.825562          -3.886203   
2           -10.063654           -11.402131          -3.886203   
3           -10.063654           -10.099939          -3.886203   
4           -10.063654           -10.602896          -3.886203   
..                 ...                  ...                ...   
94          -10.063654            -8.972545          -3.886203   
95          -10.063654            -9.071859          -3.886203   
96          -10.063654            -9.417628          -3.886203   
97          -10.063654            -9.245718          -3.886203   
98          -10.063654            -9.190990          -3.886203   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -0.957516            -0.319238   
1            -4.709132           -0.957516            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.923463           -11.631570          -3.811628   
1            -9.923463           -11.825562          -3.811628   
2            -9.923463           -11.402131          -3.811628   
3            -9.923463           -10.099939          -3.811628   
4            -9.923463           -10.602896          -3.811628   
..                 ...                  ...                ...   
94           -9.923463            -8.972545          -3.811628   
95           -9.923463            -9.071859          -3.811628   
96           -9.923463            -9.417628          -3.811628   
97           -9.923463            -9.245718          -3.811628   
98           -9.923463            -9.190990          -3.811628   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -1.105337            -0.319238   
1            -4.709132           -1.105337            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: The default dtype for empty Series will be

In [185]:
accuracy_list.count("No")

9

In [186]:
mean(range_list)

332.96

## FULL MODEL

In [187]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #slope_lin_fit_2_100
    df["cell_test_slope_lin_fit_2_100"] = pd.Series()
    df["cell_train_slope_lin_fit_2_100"] = pd.Series()
    #intercept_lin_fit_2_100
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series()
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #mean charge time
    df["cell_test_mean_charge_time"] = pd.Series()
    df["cell_train_mean_charge_time"] = pd.Series()  
    #minimum IR
    df["cell_test_minimum_IR"] = pd.Series()
    df["cell_train_minimum_IR"] = pd.Series()
    #diff_IR_100_2
    df["cell_test_diff_IR_100_2"] = pd.Series()
    df["cell_train_diff_IR_100_2"] = pd.Series()  
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_slope_lin_fit_2_100 = []
    cell_test_intercept_lin_fit_2_100 = []
    cell_test_dc2 = []
    cell_test_mean_charge_time = []
    cell_test_minimum_IR = []
    cell_test_diff_IR_100_2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_slope_lin_fit_2_100 = []
    cell_train_intercept_lin_fit_2_100 = []
    cell_train_dc2 = []
    cell_train_mean_charge_time = []
    cell_train_minimum_IR = []
    cell_train_diff_IR_100_2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_slope_lin_fit_2_100.append(x_test["slope_lin_fit_2_100"][i])
        cell_test_intercept_lin_fit_2_100.append(x_test["intercept_lin_fit_2_100"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_mean_charge_time.append(x_test["mean_charge_time"][i])
        cell_test_minimum_IR.append(x_test["minimum_IR"][i])
        cell_test_diff_IR_100_2.append(x_test["diff_IR_100_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_slope_lin_fit_2_100.append(x_train["slope_lin_fit_2_100"][a])
        cell_train_intercept_lin_fit_2_100.append(x_train["intercept_lin_fit_2_100"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_mean_charge_time.append(x_train["mean_charge_time"][a])
        cell_train_minimum_IR.append(x_train["minimum_IR"][a])
        cell_train_diff_IR_100_2.append(x_train["diff_IR_100_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_slope_lin_fit_2_100"] = pd.Series(cell_test_slope_lin_fit_2_100)
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series(cell_test_intercept_lin_fit_2_100)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_mean_charge_time"] = pd.Series(cell_test_mean_charge_time)
    df["cell_test_minimum_IR"] = pd.Series(cell_test_minimum_IR)
    df["cell_test_diff_IR_100_2"] = pd.Series(cell_test_diff_IR_100_2)


    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_slope_lin_fit_2_100"] = pd.Series(cell_train_slope_lin_fit_2_100)
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series(cell_train_intercept_lin_fit_2_100)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_mean_charge_time"] = pd.Series(cell_train_mean_charge_time)
    df["cell_train_minimum_IR"] = pd.Series(cell_train_minimum_IR)
    df["cell_train_diff_IR_100_2"] = pd.Series(cell_train_diff_IR_100_2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_slope_lin_fit_2_100"] - df["cell_train_slope_lin_fit_2_100"])**2
    d = (df["cell_test_intercept_lin_fit_2_100"] - df["cell_train_intercept_lin_fit_2_100"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_mean_charge_time"] - df["cell_train_mean_charge_time"])**2
    g = (df["cell_test_minimum_IR"] - df["cell_train_minimum_IR"])**2
    h = (df["cell_test_diff_IR_100_2"] - df["cell_train_diff_IR_100_2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f+g+h)

    #finding the 3 most similar cells
    three_most_similar_cells = df.nsmallest(3, 'distance')
    number1 = int(y_train[three_most_similar_cells.iloc[0].name])
    number2 = int(y_train[three_most_similar_cells.iloc[1].name])
    number3 = int(y_train[three_most_similar_cells.iloc[2].name])

    #getting max and min
    list_of_three_most_similar_cells = [number1, number2, number3]
    max_predicted_lifetime = max(list_of_three_most_similar_cells)
    min_predicted_lifetime = min(list_of_three_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)


/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.787515           -11.631570          -4.279456   
1           -10.787515           -11.825562          -4.279456   
2           -10.787515           -11.402131          -4.279456   
3           -10.787515           -10.099939          -4.279456   
4           -10.787515           -10.602896          -4.279456   
..                 ...                  ...                ...   
94          -10.787515            -8.972545          -4.279456   
95          -10.787515            -9.071859          -4.279456   
96          -10.787515            -9.417628          -4.279456   
97          -10.787515            -9.245718          -4.279456   
98          -10.787515            -9.190990          -4.279456   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                       -0.00001   
1            -4.709132                       -0.00001   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.315545           -11.631570          -3.566776   
1            -9.315545           -11.825562          -3.566776   
2            -9.315545           -11.402131          -3.566776   
3            -9.315545           -10.099939          -3.566776   
4            -9.315545           -10.602896          -3.566776   
..                 ...                  ...                ...   
94           -9.315545            -8.972545          -3.566776   
95           -9.315545            -9.071859          -3.566776   
96           -9.315545            -9.417628          -3.566776   
97           -9.315545            -9.245718          -3.566776   
98           -9.315545            -9.190990          -3.566776   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000016   
1            -4.709132                      -0.000016   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.325553           -11.631570          -3.553074   
1            -9.325553           -11.825562          -3.553074   
2            -9.325553           -11.402131          -3.553074   
3            -9.325553           -10.099939          -3.553074   
4            -9.325553           -10.602896          -3.553074   
..                 ...                  ...                ...   
94           -9.325553            -8.972545          -3.553074   
95           -9.325553            -9.071859          -3.553074   
96           -9.325553            -9.417628          -3.553074   
97           -9.325553            -9.245718          -3.553074   
98           -9.325553            -9.190990          -3.553074   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                       0.000004   
1            -4.709132                       0.000004   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0             -9.71285           -11.631570          -3.749464   
1             -9.71285           -11.825562          -3.749464   
2             -9.71285           -11.402131          -3.749464   
3             -9.71285           -10.099939          -3.749464   
4             -9.71285           -10.602896          -3.749464   
..                 ...                  ...                ...   
94            -9.71285            -8.972545          -3.749464   
95            -9.71285            -9.071859          -3.749464   
96            -9.71285            -9.417628          -3.749464   
97            -9.71285            -9.245718          -3.749464   
98            -9.71285            -9.190990          -3.749464   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                       -0.00002   
1            -4.709132                       -0.00002   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning

In [188]:
accuracy_list.count("No")

16

In [189]:
mean(range_list)

184.16

# APPROACH 3 (5 most similar cells)

## VARIANCE

In [190]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    df["cell_test"] = pd.Series()
    df["cell_train"] = pd.Series()
    df["distance"] = pd.Series()
    cell_test = []
    cell_train = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test.append(x_test["variance_dQ_100_10"][i])
    for a in x_train.index:
        cell_train.append(x_train["variance_dQ_100_10"][a])
    df["cell_test"] = pd.Series(cell_test)
    df["cell_train"] = pd.Series(cell_train)
    df["distance"] = abs(df["cell_test"] - df["cell_train"])

    #finding the most similar cells
    most_similar_cells = df.nsmallest(5, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")
    
    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)

#creating CSV df
csv_df = pd.DataFrame()
csv_df["cell_number_test"] = pd.Series(list(x_test.index))
csv_df["max prediction"] = pd.Series(max_predicted_lifetime_list)
csv_df["min prediction"] = pd.Series(min_predicted_lifetime_list)
csv_df["real"] = pd.Series(real_lifetime_list)
csv_df["accuracy"] = pd.Series(accuracy_list)   
csv_df.to_csv('variance_app3a.csv') 

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/e

    cell_test  cell_train  distance
0  -10.787515  -11.631570  0.844054
1  -10.787515  -11.825562  1.038046
2  -10.787515  -11.402131  0.614616
3  -10.787515  -10.099939  0.687576
4  -10.787515  -10.602896  0.184619
..        ...         ...       ...
94 -10.787515   -8.972545  1.814970
95 -10.787515   -9.071859  1.715656
96 -10.787515   -9.417628  1.369887
97 -10.787515   -9.245718  1.541797
98 -10.787515   -9.190990  1.596525

[99 rows x 3 columns]
Predicted lifetime range is from:  1078 to  2237
Actual lifetime:  1638


    cell_test  cell_train  distance
0   -10.21142  -11.631570  1.420149
1   -10.21142  -11.825562  1.614141
2   -10.21142  -11.402131  1.190711
3   -10.21142  -10.099939  0.111482
4   -10.21142  -10.602896  0.391476
..        ...         ...       ...
94  -10.21142   -8.972545  1.238875
95  -10.21142   -9.071859  1.139561
96  -10.21142   -9.417628  0.793792
97  -10.21142   -9.245718  0.965702
98  -10.21142   -9.190990  1.020430

[99 rows x 3 columns]
Predicted lifeti

In [191]:
accuracy_list.count("No")

8

In [192]:
mean(range_list)

382.48

## DISCHARGE

In [193]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #skewness
    df["cell_test_skewness"] = pd.Series()
    df["cell_train_skewness"] = pd.Series()
    #kurtosis
    df["cell_test_kurtosis"] = pd.Series()
    df["cell_train_kurtosis"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #diff discharge capacity max 2
    df["cell_test_ddc2"] = pd.Series()
    df["cell_train_ddc2"] = pd.Series()
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_skewness = []
    cell_test_kurtosis = []
    cell_test_dc2 = []
    cell_test_ddc2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_skewness = []
    cell_train_kurtosis = []
    cell_train_dc2 = []
    cell_train_ddc2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_skewness.append(x_test["skewness_dQ_100_10"][i])
        cell_test_kurtosis.append(x_test["kurtosis_dQ_100_10"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_ddc2.append(x_test["diff_discharge_capacity_max_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_skewness.append(x_train["skewness_dQ_100_10"][a])
        cell_train_kurtosis.append(x_train["kurtosis_dQ_100_10"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_ddc2.append(x_train["diff_discharge_capacity_max_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_skewness"] = pd.Series(cell_test_skewness)
    df["cell_test_kurtosis"] = pd.Series(cell_test_kurtosis)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_ddc2"] = pd.Series(cell_test_ddc2)

    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_skewness"] = pd.Series(cell_train_skewness)
    df["cell_train_kurtosis"] = pd.Series(cell_train_kurtosis)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_ddc2"] = pd.Series(cell_train_ddc2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_skewness"] - df["cell_train_skewness"])**2
    d = (df["cell_test_kurtosis"] - df["cell_train_kurtosis"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_ddc2"] - df["cell_train_ddc2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f)

    #finding the most similar cells
    most_similar_cells = df.nsmallest(5, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)

#creating CSV df
csv_df = pd.DataFrame()
csv_df["cell_number_test"] = pd.Series(list(x_test.index))
csv_df["max prediction"] = pd.Series(max_predicted_lifetime_list)
csv_df["min prediction"] = pd.Series(min_predicted_lifetime_list)
csv_df["real"] = pd.Series(real_lifetime_list)
csv_df["accuracy"] = pd.Series(accuracy_list)   
csv_df.to_csv('discharge_app3a.csv') 

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.787515           -11.631570          -4.279456   
1           -10.787515           -11.825562          -4.279456   
2           -10.787515           -11.402131          -4.279456   
3           -10.787515           -10.099939          -4.279456   
4           -10.787515           -10.602896          -4.279456   
..                 ...                  ...                ...   
94          -10.787515            -8.972545          -4.279456   
95          -10.787515            -9.071859          -4.279456   
96          -10.787515            -9.417628          -4.279456   
97          -10.787515            -9.245718          -4.279456   
98          -10.787515            -9.190990          -4.279456   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -2.117039            -0.319238   
1            -4.709132           -2.117039            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.514009           -11.631570          -3.616309   
1            -9.514009           -11.825562          -3.616309   
2            -9.514009           -11.402131          -3.616309   
3            -9.514009           -10.099939          -3.616309   
4            -9.514009           -10.602896          -3.616309   
..                 ...                  ...                ...   
94           -9.514009            -8.972545          -3.616309   
95           -9.514009            -9.071859          -3.616309   
96           -9.514009            -9.417628          -3.616309   
97           -9.514009            -9.245718          -3.616309   
98           -9.514009            -9.190990          -3.616309   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901            -1.12225            -0.319238   
1            -4.709132            -1.12225            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy


Predicted lifetime range is from:  828 to  1078
Actual lifetime:  1158


    cell_test_variance  cell_train_variance  cell_test_minimum  \
0             -9.71285           -11.631570          -3.749464   
1             -9.71285           -11.825562          -3.749464   
2             -9.71285           -11.402131          -3.749464   
3             -9.71285           -10.099939          -3.749464   
4             -9.71285           -10.602896          -3.749464   
..                 ...                  ...                ...   
94            -9.71285            -8.972545          -3.749464   
95            -9.71285            -9.071859          -3.749464   
96            -9.71285            -9.417628          -3.749464   
97            -9.71285            -9.245718          -3.749464   
98            -9.71285            -9.190990          -3.749464   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -1.217557            -0.319238   

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be

In [194]:
accuracy_list.count("No")

9

In [195]:
mean(range_list)

371.44

## FULL MODEL

In [196]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #slope_lin_fit_2_100
    df["cell_test_slope_lin_fit_2_100"] = pd.Series()
    df["cell_train_slope_lin_fit_2_100"] = pd.Series()
    #intercept_lin_fit_2_100
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series()
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #mean charge time
    df["cell_test_mean_charge_time"] = pd.Series()
    df["cell_train_mean_charge_time"] = pd.Series()  
    #minimum IR
    df["cell_test_minimum_IR"] = pd.Series()
    df["cell_train_minimum_IR"] = pd.Series()
    #diff_IR_100_2
    df["cell_test_diff_IR_100_2"] = pd.Series()
    df["cell_train_diff_IR_100_2"] = pd.Series()  
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_slope_lin_fit_2_100 = []
    cell_test_intercept_lin_fit_2_100 = []
    cell_test_dc2 = []
    cell_test_mean_charge_time = []
    cell_test_minimum_IR = []
    cell_test_diff_IR_100_2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_slope_lin_fit_2_100 = []
    cell_train_intercept_lin_fit_2_100 = []
    cell_train_dc2 = []
    cell_train_mean_charge_time = []
    cell_train_minimum_IR = []
    cell_train_diff_IR_100_2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_slope_lin_fit_2_100.append(x_test["slope_lin_fit_2_100"][i])
        cell_test_intercept_lin_fit_2_100.append(x_test["intercept_lin_fit_2_100"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_mean_charge_time.append(x_test["mean_charge_time"][i])
        cell_test_minimum_IR.append(x_test["minimum_IR"][i])
        cell_test_diff_IR_100_2.append(x_test["diff_IR_100_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_slope_lin_fit_2_100.append(x_train["slope_lin_fit_2_100"][a])
        cell_train_intercept_lin_fit_2_100.append(x_train["intercept_lin_fit_2_100"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_mean_charge_time.append(x_train["mean_charge_time"][a])
        cell_train_minimum_IR.append(x_train["minimum_IR"][a])
        cell_train_diff_IR_100_2.append(x_train["diff_IR_100_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_slope_lin_fit_2_100"] = pd.Series(cell_test_slope_lin_fit_2_100)
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series(cell_test_intercept_lin_fit_2_100)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_mean_charge_time"] = pd.Series(cell_test_mean_charge_time)
    df["cell_test_minimum_IR"] = pd.Series(cell_test_minimum_IR)
    df["cell_test_diff_IR_100_2"] = pd.Series(cell_test_diff_IR_100_2)


    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_slope_lin_fit_2_100"] = pd.Series(cell_train_slope_lin_fit_2_100)
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series(cell_train_intercept_lin_fit_2_100)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_mean_charge_time"] = pd.Series(cell_train_mean_charge_time)
    df["cell_train_minimum_IR"] = pd.Series(cell_train_minimum_IR)
    df["cell_train_diff_IR_100_2"] = pd.Series(cell_train_diff_IR_100_2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_slope_lin_fit_2_100"] - df["cell_train_slope_lin_fit_2_100"])**2
    d = (df["cell_test_intercept_lin_fit_2_100"] - df["cell_train_intercept_lin_fit_2_100"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_mean_charge_time"] - df["cell_train_mean_charge_time"])**2
    g = (df["cell_test_minimum_IR"] - df["cell_train_minimum_IR"])**2
    h = (df["cell_test_diff_IR_100_2"] - df["cell_train_diff_IR_100_2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f+g+h)

    #finding the most similar cells
    most_similar_cells = df.nsmallest(5, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)

#creating CSV df
csv_df = pd.DataFrame()
csv_df["cell_number_test"] = pd.Series(list(x_test.index))
csv_df["max prediction"] = pd.Series(max_predicted_lifetime_list)
csv_df["min prediction"] = pd.Series(min_predicted_lifetime_list)
csv_df["real"] = pd.Series(real_lifetime_list)
csv_df["accuracy"] = pd.Series(accuracy_list)   
csv_df.to_csv('fullmodel_app3a.csv') 

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.787515           -11.631570          -4.279456   
1           -10.787515           -11.825562          -4.279456   
2           -10.787515           -11.402131          -4.279456   
3           -10.787515           -10.099939          -4.279456   
4           -10.787515           -10.602896          -4.279456   
..                 ...                  ...                ...   
94          -10.787515            -8.972545          -4.279456   
95          -10.787515            -9.071859          -4.279456   
96          -10.787515            -9.417628          -4.279456   
97          -10.787515            -9.245718          -4.279456   
98          -10.787515            -9.190990          -4.279456   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                       -0.00001   
1            -4.709132                       -0.00001   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.315545           -11.631570          -3.566776   
1            -9.315545           -11.825562          -3.566776   
2            -9.315545           -11.402131          -3.566776   
3            -9.315545           -10.099939          -3.566776   
4            -9.315545           -10.602896          -3.566776   
..                 ...                  ...                ...   
94           -9.315545            -8.972545          -3.566776   
95           -9.315545            -9.071859          -3.566776   
96           -9.315545            -9.417628          -3.566776   
97           -9.315545            -9.245718          -3.566776   
98           -9.315545            -9.190990          -3.566776   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000016   
1            -4.709132                      -0.000016   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.540689           -11.631570          -3.651003   
1            -9.540689           -11.825562          -3.651003   
2            -9.540689           -11.402131          -3.651003   
3            -9.540689           -10.099939          -3.651003   
4            -9.540689           -10.602896          -3.651003   
..                 ...                  ...                ...   
94           -9.540689            -8.972545          -3.651003   
95           -9.540689            -9.071859          -3.651003   
96           -9.540689            -9.417628          -3.651003   
97           -9.540689            -9.245718          -3.651003   
98           -9.540689            -9.190990          -3.651003   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000003   
1            -4.709132                      -0.000003   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.755375           -11.631570          -3.747247   
1            -9.755375           -11.825562          -3.747247   
2            -9.755375           -11.402131          -3.747247   
3            -9.755375           -10.099939          -3.747247   
4            -9.755375           -10.602896          -3.747247   
..                 ...                  ...                ...   
94           -9.755375            -8.972545          -3.747247   
95           -9.755375            -9.071859          -3.747247   
96           -9.755375            -9.417628          -3.747247   
97           -9.755375            -9.245718          -3.747247   
98           -9.755375            -9.190990          -3.747247   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000022   
1            -4.709132                      -0.000022   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: The default dtype for empty Series will be

In [197]:
accuracy_list.count("No")

12

In [198]:
mean(range_list)

327.52

# APPROACH 4 (10 most similar cells)

## VARIANCE

In [199]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    df["cell_test"] = pd.Series()
    df["cell_train"] = pd.Series()
    df["distance"] = pd.Series()
    cell_test = []
    cell_train = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test.append(x_test["variance_dQ_100_10"][i])
    for a in x_train.index:
        cell_train.append(x_train["variance_dQ_100_10"][a])
    df["cell_test"] = pd.Series(cell_test)
    df["cell_train"] = pd.Series(cell_train)
    df["distance"] = abs(df["cell_test"] - df["cell_train"])

    #finding the most similar cells
    most_similar_cells = df.nsmallest(10, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])
    number6 = int(y_train[most_similar_cells.iloc[5].name])
    number7 = int(y_train[most_similar_cells.iloc[6].name])
    number8 = int(y_train[most_similar_cells.iloc[7].name])
    number9 = int(y_train[most_similar_cells.iloc[8].name])
    number10 = int(y_train[most_similar_cells.iloc[9].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5, number6, number7, number8, number9, number10]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)

    cell_test  cell_train  distance
0  -10.787515  -11.631570  0.844054
1  -10.787515  -11.825562  1.038046
2  -10.787515  -11.402131  0.614616
3  -10.787515  -10.099939  0.687576
4  -10.787515  -10.602896  0.184619
..        ...         ...       ...
94 -10.787515   -8.972545  1.814970
95 -10.787515   -9.071859  1.715656
96 -10.787515   -9.417628  1.369887
97 -10.787515   -9.245718  1.541797
98 -10.787515   -9.190990  1.596525

[99 rows x 3 columns]
Predicted lifetime range is from:  1009 to  2237
Actual lifetime:  1638


    cell_test  cell_train  distance
0   -10.21142  -11.631570  1.420149
1   -10.21142  -11.825562  1.614141
2   -10.21142  -11.402131  1.190711
3   -10.21142  -10.099939  0.111482
4   -10.21142  -10.602896  0.391476
..        ...         ...       ...
94  -10.21142   -8.972545  1.238875
95  -10.21142   -9.071859  1.139561
96  -10.21142   -9.417628  0.793792
97  -10.21142   -9.245718  0.965702
98  -10.21142   -9.190990  1.020430

[99 rows x 3 columns]
Predicted lifeti

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/e

In [200]:
accuracy_list.count("No")

6

In [201]:
mean(range_list)

571.36

## DISCHARGE

In [202]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #skewness
    df["cell_test_skewness"] = pd.Series()
    df["cell_train_skewness"] = pd.Series()
    #kurtosis
    df["cell_test_kurtosis"] = pd.Series()
    df["cell_train_kurtosis"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #diff discharge capacity max 2
    df["cell_test_ddc2"] = pd.Series()
    df["cell_train_ddc2"] = pd.Series()
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_skewness = []
    cell_test_kurtosis = []
    cell_test_dc2 = []
    cell_test_ddc2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_skewness = []
    cell_train_kurtosis = []
    cell_train_dc2 = []
    cell_train_ddc2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_skewness.append(x_test["skewness_dQ_100_10"][i])
        cell_test_kurtosis.append(x_test["kurtosis_dQ_100_10"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_ddc2.append(x_test["diff_discharge_capacity_max_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_skewness.append(x_train["skewness_dQ_100_10"][a])
        cell_train_kurtosis.append(x_train["kurtosis_dQ_100_10"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_ddc2.append(x_train["diff_discharge_capacity_max_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_skewness"] = pd.Series(cell_test_skewness)
    df["cell_test_kurtosis"] = pd.Series(cell_test_kurtosis)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_ddc2"] = pd.Series(cell_test_ddc2)

    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_skewness"] = pd.Series(cell_train_skewness)
    df["cell_train_kurtosis"] = pd.Series(cell_train_kurtosis)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_ddc2"] = pd.Series(cell_train_ddc2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_skewness"] - df["cell_train_skewness"])**2
    d = (df["cell_test_kurtosis"] - df["cell_train_kurtosis"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_ddc2"] - df["cell_train_ddc2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f)

    #finding the most similar cells
    most_similar_cells = df.nsmallest(10, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])
    number6 = int(y_train[most_similar_cells.iloc[5].name])
    number7 = int(y_train[most_similar_cells.iloc[6].name])
    number8 = int(y_train[most_similar_cells.iloc[7].name])
    number9 = int(y_train[most_similar_cells.iloc[8].name])
    number10 = int(y_train[most_similar_cells.iloc[9].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5, number6, number7, number8, number9, number10]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.787515           -11.631570          -4.279456   
1           -10.787515           -11.825562          -4.279456   
2           -10.787515           -11.402131          -4.279456   
3           -10.787515           -10.099939          -4.279456   
4           -10.787515           -10.602896          -4.279456   
..                 ...                  ...                ...   
94          -10.787515            -8.972545          -4.279456   
95          -10.787515            -9.071859          -4.279456   
96          -10.787515            -9.417628          -4.279456   
97          -10.787515            -9.245718          -4.279456   
98          -10.787515            -9.190990          -4.279456   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -2.117039            -0.319238   
1            -4.709132           -2.117039            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.692178           -11.631570          -3.706722   
1            -9.692178           -11.825562          -3.706722   
2            -9.692178           -11.402131          -3.706722   
3            -9.692178           -10.099939          -3.706722   
4            -9.692178           -10.602896          -3.706722   
..                 ...                  ...                ...   
94           -9.692178            -8.972545          -3.706722   
95           -9.692178            -9.071859          -3.706722   
96           -9.692178            -9.417628          -3.706722   
97           -9.692178            -9.245718          -3.706722   
98           -9.692178            -9.190990          -3.706722   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -1.167855            -0.319238   
1            -4.709132           -1.167855            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.755375           -11.631570          -3.747247   
1            -9.755375           -11.825562          -3.747247   
2            -9.755375           -11.402131          -3.747247   
3            -9.755375           -10.099939          -3.747247   
4            -9.755375           -10.602896          -3.747247   
..                 ...                  ...                ...   
94           -9.755375            -8.972545          -3.747247   
95           -9.755375            -9.071859          -3.747247   
96           -9.755375            -9.417628          -3.747247   
97           -9.755375            -9.245718          -3.747247   
98           -9.755375            -9.190990          -3.747247   

    cell_train_minimum  cell_test_skewness  cell_train_skewness  \
0            -4.720901           -1.514591            -0.319238   
1            -4.709132           -1.514591            -1.148991   
2     

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

In [203]:
print(accuracy_list.count("No"))
print(mean(range_list))

5
544.72


## FULL MODEL

In [204]:
accuracy_list = []
range_list = []
max_predicted_lifetime_list = []
min_predicted_lifetime_list = []
real_lifetime_list = []

for i in x_test.index:
    #defining stuff
    df = pd.DataFrame()
    #variance
    df["cell_test_variance"] = pd.Series()
    df["cell_train_variance"] = pd.Series()
    #minimum
    df["cell_test_minimum"] = pd.Series()
    df["cell_train_minimum"] = pd.Series()
    #slope_lin_fit_2_100
    df["cell_test_slope_lin_fit_2_100"] = pd.Series()
    df["cell_train_slope_lin_fit_2_100"] = pd.Series()
    #intercept_lin_fit_2_100
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series()
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series()
    #discharge capacity at cycle 2
    df["cell_test_dc2"] = pd.Series()
    df["cell_train_dc2"] = pd.Series()
    #mean charge time
    df["cell_test_mean_charge_time"] = pd.Series()
    df["cell_train_mean_charge_time"] = pd.Series()  
    #minimum IR
    df["cell_test_minimum_IR"] = pd.Series()
    df["cell_train_minimum_IR"] = pd.Series()
    #diff_IR_100_2
    df["cell_test_diff_IR_100_2"] = pd.Series()
    df["cell_train_diff_IR_100_2"] = pd.Series()  
    #distance
    df["distance"] = pd.Series()

    #test lists
    cell_test_variance = []
    cell_test_minimum = []
    cell_test_slope_lin_fit_2_100 = []
    cell_test_intercept_lin_fit_2_100 = []
    cell_test_dc2 = []
    cell_test_mean_charge_time = []
    cell_test_minimum_IR = []
    cell_test_diff_IR_100_2 = []

    #train lists
    cell_train_variance = []
    cell_train_minimum = []
    cell_train_slope_lin_fit_2_100 = []
    cell_train_intercept_lin_fit_2_100 = []
    cell_train_dc2 = []
    cell_train_mean_charge_time = []
    cell_train_minimum_IR = []
    cell_train_diff_IR_100_2 = []

    #creating df
    for b in range(len(x_train.index)):
        cell_test_variance.append(x_test["variance_dQ_100_10"][i])
        cell_test_minimum.append(x_test["minimum_dQ_100_10"][i])
        cell_test_slope_lin_fit_2_100.append(x_test["slope_lin_fit_2_100"][i])
        cell_test_intercept_lin_fit_2_100.append(x_test["intercept_lin_fit_2_100"][i])
        cell_test_dc2.append(x_test["discharge_capacity_2"][i])
        cell_test_mean_charge_time.append(x_test["mean_charge_time"][i])
        cell_test_minimum_IR.append(x_test["minimum_IR"][i])
        cell_test_diff_IR_100_2.append(x_test["diff_IR_100_2"][i])

    for a in x_train.index:
        cell_train_variance.append(x_train["variance_dQ_100_10"][a])
        cell_train_minimum.append(x_train["minimum_dQ_100_10"][a])
        cell_train_slope_lin_fit_2_100.append(x_train["slope_lin_fit_2_100"][a])
        cell_train_intercept_lin_fit_2_100.append(x_train["intercept_lin_fit_2_100"][a])
        cell_train_dc2.append(x_train["discharge_capacity_2"][a])
        cell_train_mean_charge_time.append(x_train["mean_charge_time"][a])
        cell_train_minimum_IR.append(x_train["minimum_IR"][a])
        cell_train_diff_IR_100_2.append(x_train["diff_IR_100_2"][a])
    
    df["cell_test_variance"] = pd.Series(cell_test_variance)
    df["cell_test_minimum"] = pd.Series(cell_test_minimum)
    df["cell_test_slope_lin_fit_2_100"] = pd.Series(cell_test_slope_lin_fit_2_100)
    df["cell_test_intercept_lin_fit_2_100"] = pd.Series(cell_test_intercept_lin_fit_2_100)
    df["cell_test_dc2"] = pd.Series(cell_test_dc2)
    df["cell_test_mean_charge_time"] = pd.Series(cell_test_mean_charge_time)
    df["cell_test_minimum_IR"] = pd.Series(cell_test_minimum_IR)
    df["cell_test_diff_IR_100_2"] = pd.Series(cell_test_diff_IR_100_2)


    df["cell_train_variance"] = pd.Series(cell_train_variance)
    df["cell_train_minimum"] = pd.Series(cell_train_minimum)
    df["cell_train_slope_lin_fit_2_100"] = pd.Series(cell_train_slope_lin_fit_2_100)
    df["cell_train_intercept_lin_fit_2_100"] = pd.Series(cell_train_intercept_lin_fit_2_100)
    df["cell_train_dc2"] = pd.Series(cell_train_dc2)
    df["cell_train_mean_charge_time"] = pd.Series(cell_train_mean_charge_time)
    df["cell_train_minimum_IR"] = pd.Series(cell_train_minimum_IR)
    df["cell_train_diff_IR_100_2"] = pd.Series(cell_train_diff_IR_100_2)

    
    a = (df["cell_test_variance"] - df["cell_train_variance"])**2
    b = (df["cell_test_minimum"] - df["cell_train_minimum"])**2
    c = (df["cell_test_slope_lin_fit_2_100"] - df["cell_train_slope_lin_fit_2_100"])**2
    d = (df["cell_test_intercept_lin_fit_2_100"] - df["cell_train_intercept_lin_fit_2_100"])**2
    e = (df["cell_test_dc2"] - df["cell_train_dc2"])**2
    f = (df["cell_test_mean_charge_time"] - df["cell_train_mean_charge_time"])**2
    g = (df["cell_test_minimum_IR"] - df["cell_train_minimum_IR"])**2
    h = (df["cell_test_diff_IR_100_2"] - df["cell_train_diff_IR_100_2"])**2

    df["distance"] = np.sqrt(a+b+c+d+e+f+g+h)

    #finding the most similar cells
    most_similar_cells = df.nsmallest(10, 'distance')
    number1 = int(y_train[most_similar_cells.iloc[0].name])
    number2 = int(y_train[most_similar_cells.iloc[1].name])
    number3 = int(y_train[most_similar_cells.iloc[2].name])
    number4 = int(y_train[most_similar_cells.iloc[3].name])
    number5 = int(y_train[most_similar_cells.iloc[4].name])
    number6 = int(y_train[most_similar_cells.iloc[5].name])
    number7 = int(y_train[most_similar_cells.iloc[6].name])
    number8 = int(y_train[most_similar_cells.iloc[7].name])
    number9 = int(y_train[most_similar_cells.iloc[8].name])
    number10 = int(y_train[most_similar_cells.iloc[9].name])

    #getting max and min
    list_of_most_similar_cells = [number1, number2, number3, number4, number5, number6, number7, number8, number9, number10]
    max_predicted_lifetime = max(list_of_most_similar_cells)
    min_predicted_lifetime = min(list_of_most_similar_cells)
    
    #find real lifetime
    real_lifetime = int(y_test[i])

        #appending to lists
    max_predicted_lifetime_list.append(float(max_predicted_lifetime))
    min_predicted_lifetime_list.append(float(min_predicted_lifetime))
    real_lifetime_list.append(float(real_lifetime))
    
    #print
    print(df)
    print("Predicted lifetime range is from: ", min_predicted_lifetime, "to ", max_predicted_lifetime)
    print("Actual lifetime: ", real_lifetime)
    print("")
    print("")

    #EVALUATING THE MODEL
    
    if (real_lifetime >= min_predicted_lifetime) & (real_lifetime <= max_predicted_lifetime):
        accuracy_list.append("Yes")
    else:
        accuracy_list.append("No")

    #GETTING THE RANGE
    range_value = abs(max_predicted_lifetime - min_predicted_lifetime)
    range_list.append(range_value)

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.787515           -11.631570          -4.279456   
1           -10.787515           -11.825562          -4.279456   
2           -10.787515           -11.402131          -4.279456   
3           -10.787515           -10.099939          -4.279456   
4           -10.787515           -10.602896          -4.279456   
..                 ...                  ...                ...   
94          -10.787515            -8.972545          -4.279456   
95          -10.787515            -9.071859          -4.279456   
96          -10.787515            -9.417628          -4.279456   
97          -10.787515            -9.245718          -4.279456   
98          -10.787515            -9.190990          -4.279456   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                       -0.00001   
1            -4.709132                       -0.00001   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be


Predicted lifetime range is from:  713 to  1115
Actual lifetime:  1315


    cell_test_variance  cell_train_variance  cell_test_minimum  \
0            -9.754075           -11.631570          -3.745685   
1            -9.754075           -11.825562          -3.745685   
2            -9.754075           -11.402131          -3.745685   
3            -9.754075           -10.099939          -3.745685   
4            -9.754075           -10.602896          -3.745685   
..                 ...                  ...                ...   
94           -9.754075            -8.972545          -3.745685   
95           -9.754075            -9.071859          -3.745685   
96           -9.754075            -9.417628          -3.745685   
97           -9.754075            -9.245718          -3.745685   
98           -9.754075            -9.190990          -3.745685   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000029   
1            -4.709

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

...

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

    cell_test_variance  cell_train_variance  cell_test_minimum  \
0             -8.80387           -11.631570          -3.317319   
1             -8.80387           -11.825562          -3.317319   
2             -8.80387           -11.402131          -3.317319   
3             -8.80387           -10.099939          -3.317319   
4             -8.80387           -10.602896          -3.317319   
..                 ...                  ...                ...   
94            -8.80387            -8.972545          -3.317319   
95            -8.80387            -9.071859          -3.317319   
96            -8.80387            -9.417628          -3.317319   
97            -8.80387            -9.245718          -3.317319   
98            -8.80387            -9.190990          -3.317319   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000009   
1            -4.709132                      -0.000009   
2            -4.575223              

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy


Predicted lifetime range is from:  713 to  1115
Actual lifetime:  1935


    cell_test_variance  cell_train_variance  cell_test_minimum  \
0           -10.230704           -11.631570          -4.031907   
1           -10.230704           -11.825562          -4.031907   
2           -10.230704           -11.402131          -4.031907   
3           -10.230704           -10.099939          -4.031907   
4           -10.230704           -10.602896          -4.031907   
..                 ...                  ...                ...   
94          -10.230704            -8.972545          -4.031907   
95          -10.230704            -9.071859          -4.031907   
96          -10.230704            -9.417628          -4.031907   
97          -10.230704            -9.245718          -4.031907   
98          -10.230704            -9.190990          -4.031907   

    cell_train_minimum  cell_test_slope_lin_fit_2_100  \
0            -4.720901                      -0.000007   
1            -4.709

/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/Users/farihahahmed/opt/miniconda3/envs/battery_project/lib/python3.7/site-packages/ipy

In [205]:
print(accuracy_list.count("No"))
print(mean(range_list))

7
475.4
